In [1]:
import re
import tweepy
import pandas as pd
from textblob import TextBlob
from sklearn import preprocessing
# from pprint import pprint
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

In [2]:
def write_from_user_tweets_to_file(screen_name):

    print('Writing to file for '+screen_name)
    
    # fetch tweets
    tweets = api.user_timeline(screen_name, count=200)
    
    try:
        
        # open file
        with open('from_user_tweets.csv', 'a') as myfile:
                
            # loop through tweets
            for tweet in tweets:
                
                # check if tweet in file
                if str(tweet.id) not in open('from_user_tweets.csv').read():
                                        
                    s = '{},{},{},{},{},{}\n'.format(tweet.id,
                                                     tweet.created_at,
                                                     tweet.favorite_count,
                                                     tweet.retweet_count,
                                                     tweet.user.screen_name, 
                                                     tweet.user.followers_count)        
                    # write to file
                    myfile.write(s)
                
    except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
            
def get_df_FROM_user_tweets():
    
    # read in data
    from_user_df = pd.read_csv('from_user_tweets.csv', header=None, engine='python')

    # name columns
    from_user_df.columns = ['tweet_id','created_at','favorite_count','retweet_count','screen_name','followers_count']

    # set tweet_id as index
    from_user_df.set_index('tweet_id', inplace=True)

    # remove duplicate tweets
    from_user_df = from_user_df[~from_user_df.index.duplicated(keep='first')]
    
    return from_user_df

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def write_about_user_tweets_to_file(screen_name):
    
    print('Writing to file for '+screen_name)
    
    # fetch tweets
    tweets = api.search(screen_name, lang='en', count=50)
    
    try:
        
        # open file
        with open('about_user_tweets.csv', 'a') as myfile:
                
            # loop through tweets
            for tweet in tweets:
                
                clean_text = clean_tweet(tweet.text)
                analysis = TextBlob(clean_text)
                polarity = analysis.sentiment.polarity
                
                # make sure tweet not already in file
                if str(tweet.id) not in open('about_user_tweets.csv').read():
                                        
                    s = '{},{},{},{},{},{},{},{}\n'.format(tweet.id, 
                                                        tweet.created_at, 
                                                        clean_text, 
                                                        polarity, 
                                                        polarity*(0.5*tweet.favorite_count+tweet.retweet_count),
                                                        tweet.favorite_count, 
                                                        tweet.retweet_count, 
                                                        screen_name)        
                    # write to file
                    myfile.write(s)
                
    except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
    
def get_df_ABOUT_user_tweets():
    
    # read in data
    about_user_df = pd.read_csv('about_user_tweets.csv', header=None)

    # name columns
    about_user_df.columns = ['tweet_id','created_at','text','polarity_ind','polarity_agg','favorite_count','retweet_count','screen_name']

    # set tweet_id as index
    about_user_df.set_index('tweet_id', inplace=True)

    # remove duplicate tweets
    about_user_df = about_user_df[~about_user_df.index.duplicated(keep='first')]

    return about_user_df

In [4]:
def ternary_plot(df):

    data = [{ 
        'type': 'scatterternary',
        'mode': 'markers',
        'opacity': .5,
        'a': df['retweet_count'],
        'b': df['rfr'],
        'c': df['favorite_count'],
        'text': df['screen_name'],
        'marker': {
            'symbol': 'o',
            'color': df['screen_name_ind'],
            'autocolorscale': True,
            'size': 12
        }
    }]

    layout = {
        'ternary': {
            'sum': 1,
            'aaxis': makeAxis('Retweets'),
            'baxis': makeAxis('<br>RFR'),
            'caxis': makeAxis('<br>Favorites')
        },
        'annotations': [{
            'showarrow': False,
            'text': 'Celebrities',
            'x': 0.5,
            'y': 1.3,
            'font': { 'size': 35 }
        }],
    }

    fig = {'data': data, 'layout': layout}
    offline.iplot(fig, validate=False)
    
def makeAxis(title): 
    return {
      'title': title,
      'titlefont': { 'size': 20 },
      'tickfont': { 'size': 15 },
      'tickcolor': 'rgba(0,0,0,0)',
      'ticklen': 5,
      'showline': True,
      'showgrid': True
    }

### Tweepy Setup

In [5]:
consumer_key = 'n9LMcL7CRMtaTY5TXMp1VfIKo'
consumer_secret = 'G0ghn8E8TJPCKl29AfmA4019U1hq6NhGQFoMsJ05CARnmkeE7U'
access_token = '1959972582-gfpDYaAbKj7c412HOalcL0jQv0QdhJtgwZguXjl'
access_token_secret = 'nZJVEdDSHsCZvV8dvRtXBjOoDIzeKOSKyvtaavjeV5ARK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api  = tweepy.API(auth)

# Tweets FROM User

In [6]:
celeb_screen_names = ['TheRock','tomhanks','prattprattpratt','LeoDiCaprio','jimmyfallon',
                      'HereIsGina','EmmaWatson','AnnaKendrick47','TheEllenShow','Oprah',
                      'Caitlyn_Jenner','DrOz','piersmorgan','charliesheen','KevinSpacey',
                      'chelseahandler','lindsaylohan','KimKardashian','GwynethPaltrow','_KrisJStewart']

In [7]:
[write_from_user_tweets_to_file(screen_name) for screen_name in celeb_screen_names]

Writing to file for TheRock
Writing to file for tomhanks
Writing to file for prattprattpratt
Writing to file for LeoDiCaprio
Writing to file for jimmyfallon
Writing to file for HereIsGina
Writing to file for EmmaWatson
Writing to file for AnnaKendrick47
Writing to file for TheEllenShow
Writing to file for Oprah
Writing to file for Caitlyn_Jenner
Writing to file for DrOz
Writing to file for piersmorgan
Writing to file for charliesheen
Writing to file for KevinSpacey
Writing to file for chelseahandler
Writing to file for lindsaylohan
Writing to file for KimKardashian
Writing to file for GwynethPaltrow
Writing to file for _KrisJStewart


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
from_user_df = get_df_FROM_user_tweets()

In [14]:
from_user_df.tail()

,created_at,favorite_count,retweet_count,screen_name,followers_count
tweet_id,,,,,
977005511743651840,2018-03-23 02:14:00,2913,162,KimKardashian,59289128
977005028454973440,2018-03-23 02:12:05,1414,62,KimKardashian,59289128
976999298956079104,2018-03-23 01:49:19,1504,79,KimKardashian,59289128
977285470261452800,2018-03-23 20:46:27,0,89,HereIsGina,459677
977287057352540160,2018-03-23 20:52:46,293,33,TheEllenShow,77296164


### Feature Engineering

In [15]:
# RFR
from_user_df['rfr'] = (from_user_df['favorite_count']+from_user_df['retweet_count'])/from_user_df['followers_count']

In [16]:
# screen_name index for plotting colors
le = preprocessing.LabelEncoder()
le.fit(from_user_df['screen_name'])
from_user_df['screen_name_ind'] = le.transform(from_user_df['screen_name'])

In [17]:
# normalize
tmp=from_user_df.drop(['created_at','screen_name'],axis=1)
normalized=(tmp-tmp.mean())/tmp.std()
normalized['screen_name']=from_user_df['screen_name']

### Plotting

In [18]:
ternary_plot(normalized)

In [19]:
tmp.index

Int64Index([976629797190213637, 976573827500003328, 976538135071899648,
            976536545472606208, 976535580656812032, 976534964228378624,
            976314388062744576, 976313171051859968, 976238163202007041,
            976237611290259457,
            ...
            977027521785905153, 977026747232223232, 977024257480732673,
            977024163838767104, 977022282991194112, 977005511743651840,
            977005028454973440, 976999298956079104, 977285470261452800,
            977287057352540160],
           dtype='int64', name='tweet_id', length=3926)

In [20]:
tmp = normalized.groupby('screen_name').mean()
tmp['screen_name'] = tmp.index
tmp
ternary_plot(tmp)

# Tweets ABOUT User

In [13]:
[write_about_user_tweets_to_file(screen_name) for screen_name in celeb_screen_names]

Writing to file for TheRock
Writing to file for tomhanks
Writing to file for prattprattpratt
Writing to file for LeoDiCaprio
Writing to file for jimmyfallon
Writing to file for HereIsGina
Writing to file for EmmaWatson
Writing to file for AnnaKendrick47
Writing to file for TheEllenShow
Writing to file for Oprah
Writing to file for Caitlyn_Jenner
Writing to file for DrOz
Writing to file for piersmorgan
Writing to file for charliesheen
Writing to file for KevinSpacey
Writing to file for chelseahandler
Writing to file for lindsaylohan
Writing to file for KimKardashian
Writing to file for GwynethPaltrow
Writing to file for _KrisJStewart


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
about_user_df = get_df_ABOUT_user_tweets()

In [ ]:
about_user_df.groupby('screen_name').mean()